In [13]:
import pandas as pd
df = pd.read_csv("SeperateLabledScoredTweets.csv")
df = df.drop(['Unnamed: 0'],axis=1)
df

,Date,Content,Username,Target,Bull Score,Bear Score,Score
0,2023-03-10,Is Fed Chair Powell “willing to take a recessi...,YahooFinance,Down,0,12,-12
1,2023-03-24,"JPMORGAN SHARES DOWN 1.1%, MORGAN STANLEY DOWN...",FirstSquawk,Up,0,12,-12
2,2023-04-19,SHARES OF CRYPTO COS DOWN AFTER BITCOIN NEARS ...,FirstSquawk,Up,0,10,-10
3,2023-03-24,"⚠️ COINBASE GLOBAL SHARES DOWN 3.6%, BITFARMS...",PriapusIQ,Up,0,10,-10
4,2023-04-13,"FOMC: ""The Fed staff now expects the economy t...",zerohedge,Up,3,13,-10
...,...,...,...,...,...,...,...
98416,2023-01-05,FED'S BULLARD: JOB MARKET CAN REMAIN RESILIENT...,FirstSquawk,Down,16,3,13
98417,2023-04-13,SHARES OF CRYPTO STOCKS UP AFTER BITCOIN CLIMB...,DeItaone,Up,14,0,14
98418,2023-04-04,Soaring technology stocks run the risk of rall...,business,Down,14,0,14
98419,2023-03-02,• Stocks open mixed this morning with sectors ...,YahooFinance,Up,17,2,15


In [14]:
#turning Target to binaary 
b_df = df.copy()
b_df['bTarget'] = b_df['Target'].map({'Up': 1, 'Down': 0})
b_df = b_df.drop('Target',axis=1)

#one hot encoding of usernames
dayScore = b_df.filter(['Date','bTarget','Score','Bull Score', 'Bear Score'])
userName = b_df.filter(['Username'])
userName = pd.get_dummies(userName)
b_df = dayScore.join(userName)

#drop dates
b_df = b_df.drop(['Date'],axis=1)

b_df

,bTarget,Score,Bull Score,Bear Score,Username_DeItaone,Username_FirstSquawk,Username_PriapusIQ,Username_Reuters,Username_WSJ,Username_YahooFinance,Username_business,Username_financialjuice,Username_unusual_whales,Username_zerohedge
0,0,-12,0,12,0,0,0,0,0,1,0,0,0,0
1,1,-12,0,12,0,1,0,0,0,0,0,0,0,0
2,1,-10,0,10,0,1,0,0,0,0,0,0,0,0
3,1,-10,0,10,0,0,1,0,0,0,0,0,0,0
4,1,-10,3,13,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98416,0,13,16,3,0,1,0,0,0,0,0,0,0,0
98417,1,14,14,0,1,0,0,0,0,0,0,0,0,0
98418,0,14,14,0,0,0,0,0,0,0,1,0,0,0
98419,1,15,17,2,0,0,0,0,0,1,0,0,0,0


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df.drop(['Target','Content', 'Date'],axis=1)
Y = df.filter(['Target'],axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.4,random_state=2023)

In [16]:
#binary split
bX = b_df.drop('bTarget',axis=1)
bY = b_df.filter(['bTarget'],axis=1)

bX_train, bX_test, bY_train, bY_test = train_test_split(bX,bY, test_size=0.4,random_state=2023)

In [17]:
from sklearn.naive_bayes import GaussianNB


gNB = GaussianNB().fit(bX_train,bY_train.values.ravel())
Y_pred = gNB.predict(bX_test)
RMSE = mean_squared_error(bY_test, Y_pred, squared = False)
accuracy = (Y_pred == bY_test.values.ravel()).sum() / len(bY_test)

print(f"GuassaRMSE")
print(f"Accuracy: {accuracy:.4f}")
print(f"RMSE: {RMSE:.4f}")



GuassaRMSE
Accuracy: 0.5232
RMSE: 0.6905


In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=2023).fit(bX_train,bY_train.values.ravel())

In [19]:
Y_pred = rf.predict(bX_test)
RMSE = mean_squared_error(bY_test, Y_pred, squared = False)
accuracy = (Y_pred == bY_test.values.ravel()).sum() / len(Y_test)

print(f"Random Forest:")
print(f"Accurancy: {accuracy:.4f}")
print(f"RMSE: {RMSE:.4f}")

Random Forest:
Accurancy: 0.5323
RMSE: 0.6839


In [20]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=2023).fit(bX_train,bY_train.values.ravel())

In [21]:
Y_pred = logreg.predict(bX_test)
RMSE = mean_squared_error(bY_test, Y_pred, squared = False)
accuracy = (Y_pred == bY_test.values.ravel()).sum() / len(bY_test)

print(f"Logistic Regression:")
print(f"Accurancy: {accuracy:.4f}")
print(f"RMSE: {RMSE:.4f}")

Logistic Regression:
Accurancy: 0.5320
RMSE: 0.6841


In [22]:
from sklearn.tree import DecisionTreeClassifier
Dtree = DecisionTreeClassifier(random_state=2023).fit(bX_train, bY_train.values.ravel())

In [23]:
Y_pred = Dtree.predict(bX_test)
RMSE = mean_squared_error(bY_test, Y_pred, squared = False)
accuracy = (Y_pred == bY_test.values.ravel()).sum() / len(Y_test)

print(f"Decision Tree:")
print(f"Accurancy: {accuracy:.4f}")
print(f"RMSE: {RMSE:.4f}")

Decision Tree:
Accurancy: 0.5320
RMSE: 0.6841
